In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alien-translation/test_no_reference
/kaggle/input/alien-translation/val
/kaggle/input/alien-translation/train


In [17]:

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW, Adam
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from transformers import PreTrainedTokenizerFast
from nltk.translate.bleu_score import sentence_bleu
from torch.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [40]:
!pip install progressbar2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.8 MB/s eta 0:00:00


In [41]:
from progressbar import ProgressBar

tqdm = ProgressBar()

In [46]:
def load_jsonl(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    tqdm = ProgressBar()
    data = [pd.json_normalize(eval(line)) for line in tqdm(lines)]
    return pd.concat(data, ignore_index=True)

In [47]:
train_df_full = load_jsonl('/kaggle/input/alien-translation/train')
val_df = load_jsonl('/kaggle/input/alien-translation/val')

100% (300000 of 300000) |################| Elapsed Time: 0:01:02 Time:  0:01:020005
100% (500 of 500) |######################| Elapsed Time: 0:00:00 Time:  0:00:000000


FileNotFoundError: [Errno 2] No such file or directory: 'kaggle/input/alien-translation/test_no_reference'

In [51]:
test_df = load_jsonl('/kaggle/input/alien-translation/test_no_reference')

100% (1000 of 1000) |####################| Elapsed Time: 0:00:00 Time:  0:00:000:00


In [212]:
# train_df = train_df_full.sample(frac=0.1, random_state=42)
train_df = train_df_full

In [53]:
with open('train_src.txt', 'w') as file:
    file.write(train_df_full.src.to_string(index=False, header=False))

In [54]:
with open('train_dst.txt', 'w') as file:
    file.write(train_df_full.dst.to_string(index=False, header=False))

In [55]:
vocab_size = 32000

In [56]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"]
trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens=special_tokens)
tokenizer.train(["train_src.txt"], trainer=trainer)
sos_token_id = tokenizer.token_to_id("[SOS]")
eos_token_id = tokenizer.token_to_id("[EOS]")
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[SOS]:0 $A:0 [EOS]:0",
    pair=f"[SOS]:0 $A:0 [EOS]:0 $B:1 [EOS]:1",
    special_tokens=[("[SOS]", sos_token_id), ("[EOS]", eos_token_id)],
)
tokenizer.decoder = decoders.WordPiece(prefix="##")

src_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[SOS]",
    sep_token="[EOS]",
)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [57]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"]
trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens=special_tokens)
tokenizer.train(["train_dst.txt"], trainer=trainer)
sos_token_id = tokenizer.token_to_id("[SOS]")
eos_token_id = tokenizer.token_to_id("[EOS]")
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[SOS]:0 $A:0 [EOS]:0",
    pair=f"[SOS]:0 $A:0 [EOS]:0 $B:1 [EOS]:1",
    special_tokens=[("[SOS]", sos_token_id), ("[EOS]", eos_token_id)],
)
tokenizer.decoder = decoders.WordPiece(prefix="##")

dst_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[SOS]",
    sep_token="[EOS]",
)

In [213]:
class TranslationDataset(Dataset):
    def __init__(self, df, src_tokenizer, dst_tokenizer):
        self.df = df
        self.src_tokenizer = src_tokenizer
        self.dst_tokenizer = dst_tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        src = self.df.iloc[idx]['src']
        src_tokens = self.src_tokenizer(
            src, return_tensors='pt'
        ).input_ids.squeeze(0)

        if 'dst' in self.df.columns:
            dst = self.df.iloc[idx]['dst']
            dst_tokens = self.dst_tokenizer(
                dst, return_tensors='pt'
            ).input_ids.squeeze(0)
        else:
            dst_tokens = None

        return src_tokens, dst_tokens

# Создаем датасеты
train_dataset = TranslationDataset(train_df, src_tokenizer, dst_tokenizer)
val_dataset = TranslationDataset(val_df, src_tokenizer, dst_tokenizer)
test_dataset = TranslationDataset(test_df, src_tokenizer, dst_tokenizer)

In [214]:
pad_token_id = src_tokenizer.pad_token_id

In [215]:
def sort_by_length(dataset):
    tqdm = ProgressBar()
    sorted_indices = sorted(tqdm(range(len(dataset))), key=lambda i: len(dataset[i][0]))
    tqdm = ProgressBar()
    sorted_dataset = [dataset[i] for i in tqdm(sorted_indices)]
    return sorted_dataset

In [216]:
val_dataset = sort_by_length(val_dataset)

100% (500 of 500) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (500 of 500) |######################| Elapsed Time: 0:00:00 Time:  0:00:000000


In [217]:
train_dataset = sort_by_length(train_dataset)

100% (300000 of 300000) |################| Elapsed Time: 0:00:00 Time:  0:00:000000
100% (300000 of 300000) |################| Elapsed Time: 0:02:14 Time:  0:02:140110


In [218]:
def collate_fn(batch):
    src_batch, dst_batch = zip(*batch)
    src_batch = torch.nn.utils.rnn.pad_sequence(
        [seq.clone().detach() for seq in src_batch], batch_first=True, padding_value=pad_token_id
    )
    if dst_batch[0] is not None:
        dst_batch = torch.nn.utils.rnn.pad_sequence(
            [seq.clone().detach() for seq in dst_batch], batch_first=True, padding_value=pad_token_id
        )
    else:
        dst_batch = None
    return src_batch, dst_batch

# DataLoaders с оптимизациями
train_loader = DataLoader(
    train_dataset, batch_size=64, collate_fn=collate_fn
)
val_loader = DataLoader(val_dataset, batch_size=64, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn)

In [219]:
# Encoder: Однонаправленный GRU с dropout и настройкой числа слоёв
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.0):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers=num_layers,
                          dropout=dropout if num_layers > 1 else 0, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

# Decoder: Однонаправленный GRU с dropout и настройкой числа слоёв
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers=num_layers,
                          dropout=dropout if num_layers > 1 else 0, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        # Входное слово (x) превращается в эмбеддинг
        embedded = self.embedding(x.unsqueeze(1))  # Добавляем размерность для batch
        outputs, hidden = self.gru(embedded, hidden)
        logits = self.fc(outputs.squeeze(1))  # Убираем добавленное измерение
        return logits, hidden

# Seq2Seq модель
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.embedding.num_embeddings

        # Создаём тензор для хранения выходов
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        # Прогон через энкодер
        enc_outputs, hidden = self.encoder(src)

        # Начальное входное слово в декодере (токен <sos>)
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t, :] = output
            # Выбираем слово с наибольшей вероятностью
            top1 = output.argmax(1)
            # Используем teacher forcing
            input = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

In [220]:
# 3. Оптимизация обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_size = vocab_size
embed_size = 256
hidden_size = 512
num_layers = 2
dropout = 0.3

encoder = Encoder(vocab_size, embed_size, hidden_size, num_layers=num_layers, dropout=dropout)
decoder = Decoder(vocab_size, embed_size, hidden_size, num_layers=num_layers, dropout=dropout)
model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = AdamW(model.parameters(), lr=0.005, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
scaler = GradScaler('cuda')
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [221]:
loaded_state_dict = torch.load('my_model1.pth')
model.load_state_dict(loaded_state_dict)

/tmp/ipykernel_30/1525342403.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state_dict = torch.load('my_model1.pth')


<All keys matched successfully>

In [223]:
# 4. Расчет BLEU
def calculate_bleu(model, val_loader, tokenizer, device):
    model.eval()
    bleu_scores = []

    with torch.no_grad():
        for src, trg in val_loader:
            src, trg = src.to(device), trg.to(device)
            outputs = model(src, trg, teacher_forcing_ratio=0)  # Без teacher forcing
            outputs = outputs.argmax(-1)  # Извлекаем предсказания

            for i in range(len(trg)):
                pred_tokens = tokenizer.decode(outputs[i].cpu().tolist(), skip_special_tokens=True)
                trg_tokens = tokenizer.decode(trg[i].cpu().tolist(), skip_special_tokens=True)
                bleu_scores.append(sentence_bleu([trg_tokens.split()], pred_tokens.split()))

    return sum(bleu_scores) / len(bleu_scores)

In [224]:
epoch = 5

In [225]:
n_epochs = epoch

In [226]:
# 5. Тренировка
for epoch in range(n_epochs, 100):
    model.train()
    epoch_loss = 0
    teacher_forcing_ratio = max(0.5 - epoch * 0.05, 0.1)
    tqdm = ProgressBar()
    for src, trg in tqdm(train_loader):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
    
        with autocast('cuda'):
            output = model(src, trg, teacher_forcing_ratio)
            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
    
        scaler.scale(loss).backward()
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
    
        epoch_loss += loss.item()
    
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for src, trg in val_loader:
            src, trg = src.to(device), trg.to(device)
            with autocast('cuda'):
                output = model(src, trg, 0)  # Без teacher forcing на валидации
                output = output[:, 1:].reshape(-1, output.shape[-1])
                trg = trg[:, 1:].reshape(-1)
                loss = criterion(output, trg)
                val_loss += loss.item()
    
    scheduler.step(val_loss)
    bleu_score = calculate_bleu(model, val_loader, dst_tokenizer, device)
    print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, BLEU: {bleu_score:.4f}")

100% (4688 of 4688) |####################| Elapsed Time: 0:20:31 Time:  0:20:311032
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
  0% (0 of 4688) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Epoch 6, Train Loss: 4.1317, Val Loss: 6.9440, BLEU: 0.3366


100% (4688 of 4688) |####################| Elapsed Time: 0:20:34 Time:  0:20:341032
  0% (0 of 4688) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Epoch 7, Train Loss: 3.9600, Val Loss: 6.9285, BLEU: 0.3306


100% (4688 of 4688) |####################| Elapsed Time: 0:20:35 Time:  0:20:351032
  0% (0 of 4688) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

...

Epoch 17, Train Loss: 3.4436, Val Loss: 6.7512, BLEU: 0.3603


100% (4688 of 4688) |####################| Elapsed Time: 0:21:09 Time:  0:21:091035
  0% (0 of 4688) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Epoch 18, Train Loss: 3.5581, Val Loss: 6.7443, BLEU: 0.3441


 54% (2559 of 4688) |##########          | Elapsed Time: 0:08:46 ETA:   0:07:18

KeyboardInterrupt: 

In [227]:
torch.save(model.state_dict(), 'my_model2.pth')

In [228]:
lst=[]
model.eval()
with torch.no_grad():
    for src, trg in test_loader:
        src = src.to(device)
        trg = torch.tensor([[1 for i in range(50)]]).to(device)
        outputs = model(src, trg, teacher_forcing_ratio=0)  # Без teacher forcing
        outputs = outputs.argmax(-1)
        lst.append(dst_tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False))

In [229]:
def truncate_before_eos(s):
    s = s.replace('[UNK] ', '')
    s = s.replace('[UNK]', '')
    eos_index = s.find(' [EOS]')
    if eos_index != -1:
        return s[:eos_index]
    return s

In [230]:
dst_lst = list(map(truncate_before_eos, lst))

In [231]:
test_df['dst'] = dst_lst

In [232]:
test_df

,dst,src
0,"., ' ' ' t t a...",◲▦◠▦◬▦■ ◉◗▢◕◗ ◍◗▱◎ ▽◠▽▪▦◠ ◕▴◉◗▦▼▴ ◀◗◓◉◧▨ ◎▴◞◠▸...
1,' ' a a a of a..,▯▴▥ ◟◧◓▨▱◨ ◀◫◓ ◈◠◈◬■ ◉◂▼◨◐◨▦ ◠▦▦◪◞◗▦◗▦ ▽◠▢◈◬◐▪...
2,could ' ' the ' of..,◡◠▻◧▦ ◂▫◧◎◂◀◗▱ ◍◗◓◎◠◞◬ ◠▦▱◠◒◎◠◞▪▢ ◝◓▴▹◗▫ ◈◨◓▾◎...
3,went with the...,◝▾◀◀◠ ▰◠▫◞◂▦ ◚▴ ▰▴◀◀ ▮◫◎▻◞◂▦■ ◞◠◀◠▷ ◂◳▦◠▦◠▦ ◍◂...
4,"Don ' t t you you.""","""○◐▱◠◈◬◐▪▦▪ ◕◣◓◎◪▱◪◓◗▦◪ ◠◞▱◠ ◗▢◫▦ ◚▴◓◎▴■"" ◈◪◈◫..."
...,...,...
995,I it the the..,◝◠▢▴▦ ◀▾ ◈◨◓◨◎■ ▨▩◉◭▨ ◎◪◞◠◍◪▱▴◓ ◀◧▽▾▦▼◠ ◠▦◫ ◞◬...
996,: s ' s...,▯◝▬ ▯◪▥◞'◫▦ ▷◠◀▴◓◫▦◪ ◕◣◓◪ ▬◨◎▷◨◓◫▽◪▫◉◗ □◠◓▫◗ ◮...
997,In the the...,2018: ◰▦◈◂▦▴▢▽◠'▦▪▦ ▫◨◓◗◞▫◗▨ ▲◧◎◀◧▨ ◠◈◠◞◬▦◈◠ ◎...
998,",,, '....",1938'◈◪ ◀◫◓ ▫◞▾▦◠◎◫ ◙◧▦◕◕◠▱◠'◈◠ 200'◈◪▦ ◍◠▢▱◠ ...


In [233]:
new_order = ['dst', 'src']

# Переупорядочивание колонок
test_df = test_df.reindex(columns=new_order)

In [234]:
test_df

,dst,src
0,"., ' ' ' t t a...",◲▦◠▦◬▦■ ◉◗▢◕◗ ◍◗▱◎ ▽◠▽▪▦◠ ◕▴◉◗▦▼▴ ◀◗◓◉◧▨ ◎▴◞◠▸...
1,' ' a a a of a..,▯▴▥ ◟◧◓▨▱◨ ◀◫◓ ◈◠◈◬■ ◉◂▼◨◐◨▦ ◠▦▦◪◞◗▦◗▦ ▽◠▢◈◬◐▪...
2,could ' ' the ' of..,◡◠▻◧▦ ◂▫◧◎◂◀◗▱ ◍◗◓◎◠◞◬ ◠▦▱◠◒◎◠◞▪▢ ◝◓▴▹◗▫ ◈◨◓▾◎...
3,went with the...,◝▾◀◀◠ ▰◠▫◞◂▦ ◚▴ ▰▴◀◀ ▮◫◎▻◞◂▦■ ◞◠◀◠▷ ◂◳▦◠▦◠▦ ◍◂...
4,"Don ' t t you you.""","""○◐▱◠◈◬◐▪▦▪ ◕◣◓◎◪▱◪◓◗▦◪ ◠◞▱◠ ◗▢◫▦ ◚▴◓◎▴■"" ◈◪◈◫..."
...,...,...
995,I it the the..,◝◠▢▴▦ ◀▾ ◈◨◓◨◎■ ▨▩◉◭▨ ◎◪◞◠◍◪▱▴◓ ◀◧▽▾▦▼◠ ◠▦◫ ◞◬...
996,: s ' s...,▯◝▬ ▯◪▥◞'◫▦ ▷◠◀▴◓◫▦◪ ◕◣◓◪ ▬◨◎▷◨◓◫▽◪▫◉◗ □◠◓▫◗ ◮...
997,In the the...,2018: ◰▦◈◂▦▴▢▽◠'▦▪▦ ▫◨◓◗◞▫◗▨ ▲◧◎◀◧▨ ◠◈◠◞◬▦◈◠ ◎...
998,",,, '....",1938'◈◪ ◀◫◓ ▫◞▾▦◠◎◫ ◙◧▦◕◕◠▱◠'◈◠ 200'◈◪▦ ◍◠▢▱◠ ...


In [235]:
test_df.to_json('test1.jsonl', orient='records', lines=True)